In [1]:
# Define the sample HT-Seq datafiles
datadir = '/data/BIDS-HPC/private/projects/dmi2/data/all_gene_expression_files_in_target/links'
file_fpkm = 'fffee315-9aa3-44d2-8c89-78a2c1d107e7.FPKM.txt'
file_fpkm_uq = 'fffee315-9aa3-44d2-8c89-78a2c1d107e7.FPKM-UQ.txt'
file_counts = 'fffee315-9aa3-44d2-8c89-78a2c1d107e7.htseq_counts.txt'

# Define the reference datafiles
gdc_tsv_file = '/data/BIDS-HPC/private/projects/dmi2/data/gencode.gene.info.v22.tsv'
gencode_gtf_file = '/data/BIDS-HPC/private/projects/dmi2/data/gencode.v22.annotation-just_genes.gtf'

In [2]:
# Read in all the datafiles into Pandas dataframes
import pandas as pd
import os
df_fpkm = pd.read_csv(os.path.join(datadir, file_fpkm), sep='\t', names=['id','intensity'])
df_fpkm_uq = pd.read_csv(os.path.join(datadir, file_fpkm_uq), sep='\t', names=['id','intensity'])
df_count = pd.read_csv(os.path.join(datadir, file_counts), sep='\t', skipfooter=5, names=['id','intensity'])
df_gdc = pd.read_csv(gdc_tsv_file, sep='\t')
df_gencode = pd.read_csv(gencode_gtf_file, sep='\t', skiprows=5, header=None)

In [3]:
# Format the df_gencode dataframe for consistency
df_gencode['id'] = df_gencode.apply(lambda x: x[8].split()[1].split('\"')[1], axis=1)
df_gencode['type'] = df_gencode.apply(lambda x: x[8].split()[3].split('\"')[1], axis=1)
df_gencode['name'] = df_gencode.apply(lambda x: x[8].split()[7].split('\"')[1], axis=1)
df_gencode = df_gencode.rename({3: 'start', 4: 'end', 6: 'strand', 0: 'seqname'}, axis='columns')
df_gencode = df_gencode.set_index('id')
df_gencode = df_gencode.sort_index()

In [3]:
df

NameError: name 'df' is not defined

In [4]:
# Format the df_gdc dataframe for consistency
df_gdc = df_gdc.rename({'gene_id': 'id', 'gene_name': 'name', 'gene_type': 'type'}, axis='columns')
df_gdc = df_gdc.set_index('id')
df_gdc = df_gdc.sort_index()

In [5]:
# Check for column equality between the two reference datafiles
for colname in ['name', 'seqname', 'start', 'end', 'strand', 'type']:
    print(df_gdc[colname].equals(df_gencode[colname]))

True
True
True
True
True
True


In [3]:
# This line was edited but was used to show that the ID columns of all three dataframes are exactly the same
(df_fpkm_uq['id']==df_counts['id'])

0        True
1        True
2        True
3        True
4        True
         ... 
60478    True
60479    True
60480    True
60481    True
60482    True
Name: id, Length: 60483, dtype: bool

In [6]:
# Check that the ID columns of all five dataframes are exactly the same
dfs = [df_fpkm, df_fpkm_uq, df_count, df_gdc, df_gencode]
ndfs = len(dfs)
for idf in range(ndfs-1):
    print(idf)
#print(df_gdc.index.equals(df_gencode.index))

0
1
2
3


In [7]:
df_fpkm

,id,intensity
0,ENSG00000000003.13,48.2789
1,ENSG00000000005.5,4.0536
2,ENSG00000000419.11,65.1115
3,ENSG00000000457.12,1.5301
4,ENSG00000000460.15,2.9406
...,...,...
60478,ENSGR0000275287.3,0.0000
60479,ENSGR0000276543.3,0.0000
60480,ENSGR0000277120.3,0.0000
60481,ENSGR0000280767.1,0.0000


In [27]:
tol = 1e-6
((df_fpkm['fpkm']/df_fpkm['fpkm'].sum() - df_fpkm_uq['fpkm-uq']/df_fpkm_uq['fpkm-uq'].sum()).abs() < tol).sum() # This line shows that the FPKM and FPKM-UQ files have the same values up to a fixed normalization
((df_counts['count']/df_counts['count'].sum() - df_fpkm_uq['fpkm-uq']/df_fpkm_uq['fpkm-uq'].sum()).abs() < tol).sum() # this line shows that this is not true of the counts

43200

In [30]:
#set(df_fpkm['id']) - set(df_gdc['gene_id'])
set(df_fpkm['id']) == set(df_gdc['gene_id'])

True

In [68]:
import numpy as np
arr = df_fpkm['fpkm'].to_numpy()
quantiles = np.quantile(arr, [0,0.25,0.5,0.75,1])
#arr[quantiles[0]<arr and arr<quantiles[3]]
print(arr[(quantiles[3]<arr) & (arr<quantiles[4])].sum())
1/(df_fpkm['fpkm'].max()/df_fpkm_uq['fpkm-uq'].max())

331163.3667


26613.247395329272

In [45]:
arr.max()/2

3865.25345

In [29]:
pd.qcut(df_fpkm['fpkm'], 4)

ValueError: Bin edges must be unique: array([0.0000000e+00, 0.0000000e+00, 4.7900000e-02, 8.8105000e-01,
       7.7305069e+03]).
You can drop duplicate edges by setting the 'duplicates' kwarg

In [9]:
df_counts

,id,count
0,ENSG00000000003.13,11205
1,ENSG00000000005.5,334
2,ENSG00000000419.11,4022
3,ENSG00000000457.12,539
4,ENSG00000000460.15,898
...,...,...
60478,ENSGR0000275287.3,0
60479,ENSGR0000276543.3,0
60480,ENSGR0000277120.3,0
60481,ENSGR0000280767.1,0


In [83]:
#df_gdc.index.equals(df_gencode.index)
#df_gdc['name'].equals(df_gencode['name'])
#df_gdc['seqname'].equals(df_gencode['seqname'])
#df_gdc['start'].equals(df_gencode['start'])
#df_gdc['end'].equals(df_gencode['end'])
#df_gdc['strand'].equals(df_gencode['strand'])
#df_gdc['type'].equals(df_gencode['type'])

KeyError: 'id'

In [70]:
df_gdc

,name,seqname,start,end,strand,type,gene_status,havana_gene,full_length,exon_length,exon_num
id,,,,,,,,,,,
ENSG00000000003.13,TSPAN6,chrX,100627109,100639991,-,protein_coding,KNOWN,OTTHUMG00000022002.1,12883,4535,34
ENSG00000000005.5,TNMD,chrX,100584802,100599885,+,protein_coding,KNOWN,OTTHUMG00000022001.1,15084,1610,10
ENSG00000000419.11,DPM1,chr20,50934867,50958555,-,protein_coding,KNOWN,OTTHUMG00000032742.2,23689,1207,53
ENSG00000000457.12,SCYL3,chr1,169849631,169894267,-,protein_coding,KNOWN,OTTHUMG00000035941.4,44637,6883,63
ENSG00000000460.15,C1orf112,chr1,169662007,169854080,+,protein_coding,KNOWN,OTTHUMG00000035821.7,192074,5967,151
...,...,...,...,...,...,...,...,...,...,...,...
ENSGR0000275287.3,Metazoa_SRP,chrY,388100,388389,-,misc_RNA,NOVEL,NaN,290,290,1
ENSGR0000276543.3,AJ271736.1,chrY,57209151,57209218,+,miRNA,NOVEL,NaN,68,68,1
ENSGR0000277120.3,MIR6089,chrY,2609191,2609254,+,miRNA,KNOWN,NaN,64,64,1


In [71]:
df_gencode

,seqname,1,2,start,end,5,strand,7,8,type,name
id,,,,,,,,,,,
ENSG00000000003.13,chrX,HAVANA,gene,100627109,100639991,.,-,.,"gene_id ""ENSG00000000003.13""; gene_type ""prote...",protein_coding,TSPAN6
ENSG00000000005.5,chrX,HAVANA,gene,100584802,100599885,.,+,.,"gene_id ""ENSG00000000005.5""; gene_type ""protei...",protein_coding,TNMD
ENSG00000000419.11,chr20,HAVANA,gene,50934867,50958555,.,-,.,"gene_id ""ENSG00000000419.11""; gene_type ""prote...",protein_coding,DPM1
ENSG00000000457.12,chr1,HAVANA,gene,169849631,169894267,.,-,.,"gene_id ""ENSG00000000457.12""; gene_type ""prote...",protein_coding,SCYL3
ENSG00000000460.15,chr1,HAVANA,gene,169662007,169854080,.,+,.,"gene_id ""ENSG00000000460.15""; gene_type ""prote...",protein_coding,C1orf112
...,...,...,...,...,...,...,...,...,...,...,...
ENSGR0000275287.3,chrY,ENSEMBL,gene,388100,388389,.,-,.,"gene_id ""ENSGR0000275287.3""; gene_type ""misc_R...",misc_RNA,Metazoa_SRP
ENSGR0000276543.3,chrY,ENSEMBL,gene,57209151,57209218,.,+,.,"gene_id ""ENSGR0000276543.3""; gene_type ""miRNA""...",miRNA,AJ271736.1
ENSGR0000277120.3,chrY,ENSEMBL,gene,2609191,2609254,.,+,.,"gene_id ""ENSGR0000277120.3""; gene_type ""miRNA""...",miRNA,MIR6089


In [17]:
df1 = pd.DataFrame({1: [10,11], 2: [20,21]}, index=['a','b'])
df2 = pd.DataFrame({1: [11,10], 2: [21,20]}, index=['b','a'])
df2.sort_index().equals(df1)

True

In [15]:
df2.sort_index()

,1,2
a,10,20
b,11,21


In [47]:
df_gencode['id'] = df_gencode.apply(lambda x: x[8].split()[1].split('\"')[1], axis=1)
df_gencode['type'] = df_gencode.apply(lambda x: x[8].split()[3].split('\"')[1], axis=1)
df_gencode['name'] = df_gencode.apply(lambda x: x[8].split()[7].split('\"')[1], axis=1)
df_gencode = df_gencode.rename({3: 'start', 4: 'stop'}, axis='columns')
df_gencode = df_gencode.set_index('id')
df_gencode = df_gencode.sort_index()

In [48]:
df_gencode

,0,1,2,start,stop,5,6,7,8,type,name
id,,,,,,,,,,,
ENSG00000000003.13,chrX,HAVANA,gene,100627109,100639991,.,-,.,"gene_id ""ENSG00000000003.13""; gene_type ""prote...",protein_coding,TSPAN6
ENSG00000000005.5,chrX,HAVANA,gene,100584802,100599885,.,+,.,"gene_id ""ENSG00000000005.5""; gene_type ""protei...",protein_coding,TNMD
ENSG00000000419.11,chr20,HAVANA,gene,50934867,50958555,.,-,.,"gene_id ""ENSG00000000419.11""; gene_type ""prote...",protein_coding,DPM1
ENSG00000000457.12,chr1,HAVANA,gene,169849631,169894267,.,-,.,"gene_id ""ENSG00000000457.12""; gene_type ""prote...",protein_coding,SCYL3
ENSG00000000460.15,chr1,HAVANA,gene,169662007,169854080,.,+,.,"gene_id ""ENSG00000000460.15""; gene_type ""prote...",protein_coding,C1orf112
...,...,...,...,...,...,...,...,...,...,...,...
ENSGR0000275287.3,chrY,ENSEMBL,gene,388100,388389,.,-,.,"gene_id ""ENSGR0000275287.3""; gene_type ""misc_R...",misc_RNA,Metazoa_SRP
ENSGR0000276543.3,chrY,ENSEMBL,gene,57209151,57209218,.,+,.,"gene_id ""ENSGR0000276543.3""; gene_type ""miRNA""...",miRNA,AJ271736.1
ENSGR0000277120.3,chrY,ENSEMBL,gene,2609191,2609254,.,+,.,"gene_id ""ENSGR0000277120.3""; gene_type ""miRNA""...",miRNA,MIR6089
